In [1]:
import sys, json, time
from threading import Thread, Condition, Event
from psonic import *
sys.path.append("./repo/piplay")
from weather_pilock.SenseHat_Data import Sensor
from pysonic.play_weather import Notes, Rates, rescale
from helpers.helpers import *
import asyncio
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox
from IPython.display import display

golden_ratio = golden()


In [2]:
condition = Condition()
stop_event = Event()
rates = Rates()
senseHat = Sensor()

def live_producer():
    sleep(rates.seconds_per_beat)
    pass
def live_loop_producer(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.notifyAll() #Message to threads
        live_producer()
live_thread_producer = Thread(name="producer", target=live_loop_producer, args=(condition, stop_event))

def play_temp():
    pass
def live_loop_temp(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Message to threads
        play_temp()
live_thread_temp = Thread(name="consumer_temp", target=live_loop_temp, args=(condition, stop_event))

def play_pressure():
    pass    
def live_loop_pressure(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_pressure()
live_thread_pressure = Thread(name="consumer_pressure", target=live_loop_pressure, args=(condition, stop_event))

def play_humidity():
    pass
def live_loop_humidity(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_humidity()
live_thread_humidity = Thread(name="consumer_humidity", target=live_loop_humidity, args=(condition, stop_event))

In [ ]:
start_threads_button = widgets.Button(
    description='Start Threads'
)
def start_threads(b=None):
    global live_thread_producer
    live_thread_producer.start()
    global live_thread_temp
    live_thread_temp.start()
    global live_thread_pressure
    live_thread_pressure.start()
    global live_thread_humidity
    live_thread_humidity.start()     
start_threads_button.on_click(start_threads)

stop_threads_button = widgets.Button(
    description='Stop Threads'
)
def stop_threads(b=None):
    global stop_event
    stop_event.set()
stop_threads_button.on_click(stop_threads)

restart_threads_button = widgets.Button(
    description='Restart Threads'
)
def restart_threads(b=None):
    global stop_event
    stop_event = Event()
    global condition
    condition = Condition()
    global live_loop_producer
    global live_thread_producer
    live_thread_producer = Thread(name="producer", target=live_loop_producer, args=(condition, stop_event))
    live_thread_producer.start()
restart_threads_button.on_click(restart_threads)

tempo_slider = widgets.FloatSlider(value = rates.seconds_per_beat, min = 0.1, max = 2.0, step = 0.1, description="Tempo(Hz)", orientation="horizontal")
update_tempo = widgets.Button(
            description='Update Tempo'
        )
def set_tempo(b):
    global live_producer
    def live_producer():
        sleep(1 / tempo_slider.value)
        pass
update_tempo.on_click(set_tempo)

class NoteUI:
    def __init__(self):
        self.amp_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="amp", orientation="horizontal")
        self.pan_slider = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description="pan", orientation="horizontal")
        self.attack_slider = widgets.FloatSlider(value=0.1, min=0.0, max=2.0, step=0.1, description="attack", orientation="horizontal")
        self.decay_slider = widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.1, description="decay", orientation="horizontal")
        self.sustain_slider = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description="sustain", orientation="horizontal")
        self.sustain_level_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description="sustain level", orientation="vertical")
        self.release_slider = widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description="release", orientation="horizontal")
        self.play_button = widgets.Button(
            description='Update'
        )
    
temp_ui = NoteUI()
def play_temp_clicked(b):
    global play_temp
    def play_temp():
        global senseHat
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.temperature_from_humidity), 
                      rescale(senseHat.sense.temperature_from_pressure)
                     ], 
                     amp = temp_ui.amp_slider.value,
                     pan = temp_ui.pan_slider.value,
                     attack = temp_ui.attack_slider.value, 
                     decay = temp_ui.decay_slider.value,
                     sustain = temp_ui.sustain_slider.value,
                     sustain_level = temp_ui.sustain_level_slider.value,
                     release = temp_ui.release_slider.value
                    )
        play(temp.notes, amp=temp.amp, pan=temp.pan, 
             attack=temp.attack, decay=temp.decay, 
             sustain=temp.sustain, sustain_level=temp.sustain_level, 
             release=temp.release)
        pass
temp_ui.play_button.on_click(play_temp_clicked)

pressure_ui = NoteUI()
def play_pressure_clicked(b):
    global play_pressure
    def play_pressure():
        global senseHat
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.pressure)], 
                     amp = pressure_ui.amp_slider.value,
                     pan = pressure_ui.pan_slider.value,
                     attack = pressure_ui.attack_slider.value, 
                     decay = pressure_ui.decay_slider.value,
                     sustain = pressure_ui.sustain_slider.value,
                     sustain_level = pressure_ui.sustain_level_slider.value,
                     release = pressure_ui.release_slider.value
                    )
        play(temp.notes, amp=temp.amp, pan=temp.pan, 
             attack=temp.attack, decay=temp.decay, 
             sustain=temp.sustain, sustain_level=temp.sustain_level, 
             release=temp.release)
        pass
pressure_ui.play_button.on_click(play_pressure_clicked)

humidity_ui = NoteUI()
def play_humidity_clicked(b):
    global play_humidity
    def play_humidity():
        global senseHat
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.humidity)], 
                     amp = humidity_ui.amp_slider.value,
                     pan = humidity_ui.pan_slider.value,
                     attack = humidity_ui.attack_slider.value, 
                     decay = humidity_ui.decay_slider.value,
                     sustain = humidity_ui.sustain_slider.value,
                     sustain_level = humidity_ui.sustain_level_slider.value,
                     release = humidity_ui.release_slider.value
                    )
        play(temp.notes, amp=temp.amp, pan=temp.pan, 
             attack=temp.attack, decay=temp.decay, 
             sustain=temp.sustain, sustain_level=temp.sustain_level, 
             release=temp.release)
        pass
humidity_ui.play_button.on_click(play_humidity_clicked)


In [4]:
tab_controls = HBox(children=[start_threads_button, stop_threads_button, tempo_slider, update_tempo])

def build_note_tab(ui):
    return HBox(children=[VBox(children=[ui.play_button]), 
                          VBox(children=[ui.amp_slider, temp_ui.pan_slider]),
                          VBox(children=[ui.sustain_level_slider]),
                          VBox(children=[ui.attack_slider, 
                                         ui.decay_slider, 
                                         ui.sustain_slider, 
                                         ui.release_slider
                                        ]
                              )
                         ]
               )

tab_temp = build_note_tab(temp_ui)
tab_pressure = build_note_tab(pressure_ui)
tab_humidity = build_note_tab(humidity_ui)

tab_sounds = widgets.Tab(children=[tab_temp, tab_pressure, tab_humidity])
tab_sounds.set_title(0, 'Temperature')
tab_sounds.set_title(1, 'Pressure')
tab_sounds.set_title(2, 'Humidity')

VBox(children=[tab_controls, tab_sounds])


RuntimeError: threads can only be started once